In [122]:
import csv
from itertools import combinations
import pandas as pd


In [123]:
dataf = pd.read_csv("dataset2.csv")
dataf

,Watcher,videoCategoryId,videoCategoryLabel,definition
0,Billy,29,Nonprofits & Activism,hd
1,Leila,22,People & Blogs,sd
2,Billy,22,People & Blogs,sd
3,Mark,24,Entertainment,hd
4,Billy,24,Entertainment,hd
...,...,...,...,...
110,Jeff,24,Entertainment,hd
111,Billy,26,Howto & Style,sd
112,Clark,28,Science & Technology,hd
113,Billy,24,Entertainment,sd


In [124]:
df= dataf.set_index('Watcher')
df.loc['Billy']

,videoCategoryId,videoCategoryLabel,definition
Watcher,,,
Billy,29,Nonprofits & Activism,hd
Billy,22,People & Blogs,sd
Billy,24,Entertainment,hd
Billy,28,Science & Technology,hd
Billy,26,Howto & Style,sd
Billy,24,Entertainment,sd


In [125]:
df.index.unique().shape

(39,)

In [126]:
[df.loc['Flo']['videoCategoryLabel']]

['Science & Technology']

In [127]:
def organize_df(dataframe, new_index, target):
        new_df = dataframe.set_index(new_index)
        user_categs = {}
        for each in new_df.index.unique():
                if(isinstance(new_df.loc[each][target], str)): 
                        user_categs[each] = [new_df.loc[each][target]]
                else: user_categs[each] = list(set(new_df.loc[each][target]))
        return user_categs

In [128]:
organize_df(dataf, "Watcher", "videoCategoryLabel")

{'Billy': ['Howto & Style',
  'Entertainment',
  'Science & Technology',
  'Nonprofits & Activism',
  'People & Blogs'],
 'Leila': ['Science & Technology',
  'Film & Animation',
  'Comedy',
  'Nonprofits & Activism',
  'People & Blogs'],
 'Mark': ['Entertainment',
  'Science & Technology',
  'Education',
  'Nonprofits & Activism',
  'People & Blogs'],
 'Jane': ['Entertainment', 'People & Blogs', 'Education'],
 'Babs': ['People & Blogs'],
 'Jeff': ['Gaming',
  'Science & Technology',
  'Entertainment',
  'Education',
  'News & Politics'],
 'Naomy': ['Science & Technology',
  'People & Blogs',
  'Nonprofits & Activism',
  'Music'],
 'Flo': ['Science & Technology'],
 'Phoebe': ['Science & Technology', 'People & Blogs'],
 'Rachel': ['Science & Technology', 'People & Blogs'],
 'Marie': ['Entertainment', 'People & Blogs'],
 'Ross': ['News & Politics', 'Education'],
 'Adam': ['People & Blogs'],
 'Ben': ['People & Blogs'],
 'Brad': ['Science & Technology',
  'People & Blogs',
  'Nonprofits & A

In [129]:
def frequence(items_lst, trans, check=False):
    items_counts = dict()
    for i in items_lst:
        temp_i = {i}
        if check:
            temp_i = set(i)
        for j in trans.items():
            if temp_i.issubset(set(j[1])):
                if i in items_counts:
                    items_counts[i] += 1
                else:
                    items_counts[i] = 1
    return items_counts

In [130]:
def support(items_counts, trans):
    support = dict()
    total_trans = len(trans)
    for i in items_counts:
        support[i] = items_counts[i]/total_trans
    return support

In [131]:
def association_rules(items_grater_then_min_support):
    rules = []
    dict_rules = {}
    for i in items_grater_then_min_support:
        dict_rules = {}
        if type(i) != type(str()):
            i = list(i)
            temp_i = i[:]
            for j in range(len(i)):
                k = temp_i[j]
                del temp_i[j]
                dict_rules[k] = temp_i
                temp_i = i[:]
        rules.append(dict_rules)
    temp = []
    for i in rules:
        for j in i.items():
            if type(j[1]) != type(str()):
                temp.append({tuple(j[1])[0]: j[0]})
            else:
                temp.append({j[1]: j[0]})
    rules.extend(temp)
    return rules

In [132]:
def confidence(associations, d, min_confidence, N):
    ans = {}
    for i in associations:
        for j in i.items():
            if type(j[0]) == type(str()):
                left = {j[0]}
            else:
                left = set(j[0])
            if type(j[1]) == type(str()):
                right = {j[1]}
            else:
                right = set(j[1])
            for k in d:
                if type(k) != type(str()):
                    if left.union(right) - set(k) == set():
                        up = d[k]
                    if len(right) == len(set(k)) and right - set(k) == set():
                            down = d[k]
                else:
                    if len(right) >= len({k}):
                        if right - {k} == set():
                            down = d[k]
                    elif len(right) <= len({k}):
                        if {k} - right == set():
                            down = d[k]
            if up/down >= min_confidence:
                ans[tuple(left)[0]] = right, up/down, up/N, down/N
    return ans   

In [133]:
def correlation_rule(associations, d, min_lift, N):
    ans = {}
    for i in associations:
        for j in i.items():
            if type(j[0]) == type(str()):
                left = {j[0]}
            else:
                left = set(j[0])
            if type(j[1]) == type(str()):
                right = {j[1]}
            else:
                right = set(j[1])
            for k in d:
                if type(k) != type(str()):
                    if left.union(right) - set(k) == set():
                        up = d[k]
                    if len(right) == len(set(k)) and right - set(k) == set():
                            down = d[k]
                else:
                    if len(right) >= len({k}):
                        if right - {k} == set():
                            down = d[k]
                    elif len(right) <= len({k}):
                        if {k} - right == set():
                            down = d[k]
            if (up/down)/(down/N) >= min_lift:
                ans[tuple(left)[0]] = right, up/down, up/N, down/N, (up/down)/(down/N)
    return ans   

In [134]:
def apriori(file_loc,min_support=0.01, min_confidence=0.7, min_lift=1):
    df = pd.read_csv(file_loc)
    trans = organize_df(df, "Watcher", "videoCategoryLabel")
    
    number_of_trans = [len(i) for i in trans.values()]
    items_lst = set()
    
    itemcount_track = list()    
    
    for i in trans.values():
        for j in i:
            items_lst.add(j)
    
    store_item_lst = list(items_lst)[:]
    items_grater_then_min_support = list()
    items_counts = frequence(items_lst, trans)
    itemcount_track.append(items_counts)
    items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})
    
    for i in range(2, max(number_of_trans)+1):
        item_list = combinations(items_lst, i)
        items_counts = frequence(item_list, trans, check=True)
        itemcount_track.append(items_counts)
        if list({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support}.keys()) != []:
            items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})
        
    d = {}
    {d.update(i) for i in itemcount_track}
    associations = association_rules(items_grater_then_min_support[len(items_grater_then_min_support)-1])
    associations_grater_then_confidence = confidence(associations, d, min_confidence, len(trans.keys()))
    correlation_rules = correlation_rule(associations,d, min_lift,len(trans.keys()))
    return associations_grater_then_confidence, correlation_rules

In [135]:
res = apriori('dataset2.csv',0.01, 0.7, 0.2)
#pd.DataFrame.from_dict(res[0], orient='index', columns=['association_rule','confidence', 'support_asso12', 'support_asso1'])
pd.DataFrame.from_dict(res[1], orient='index', columns=['association_rule','confidence', 'support_asso12', 'support_asso1' ,'lift'])

,association_rule,confidence,support_asso12,support_asso1,lift
Science & Technology,{News & Politics},0.2,0.025641,0.128205,1.56
Nonprofits & Activism,"{Science & Technology, Education, Sports, News...",1.0,0.025641,0.025641,39.00
Education,"{Science & Technology, Nonprofits & Activism, ...",1.0,0.025641,0.025641,39.00
Sports,"{Science & Technology, Education, Nonprofits &...",1.0,0.025641,0.025641,39.00
News & Politics,"{Science & Technology, Education, Nonprofits &...",1.0,0.025641,0.025641,39.00
People & Blogs,"{Science & Technology, Education, Nonprofits &...",1.0,0.025641,0.025641,39.00
